# Explorando bases vetoriais

In [1]:
from langchain.document_loaders import PyPDFLoader

pdf_path = 'data/PPC_TSI_EaD.pdf'
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    #chunk_size=200,
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
)

texts = text_splitter.split_documents(pages)

print(texts[0])

page_content='Ministério da Educação
Instituto Federal de Educação, Ciência e Tecnologia do Rio Grande do Norte
DELIBERAÇÃO Nº 59/2022 - CONSEPEX/IFRN DELIBERAÇÃO Nº 59/2022 - CONSEPEX/IFRN   
   21 de dezembro de 2022    21 de dezembro de 2022
O PRESIDENTE DO CONSELHO DE ENSINO, PESQUISA E EXTE NSÃO DO INSTITUTO FEDERAL DE
EDUCAÇÃO, CIÊNCIA E TECNOLOGIA DO RIO GRANDE DO NOR TE, faz saber que este Conselho, reunido
extraordinariamente, de forma híbrida, em 18 de novembro de 2022, no uso das at ribuições que lhe confere o art. 13 do Estatuto do
IFRN, e
CONSIDERANDO
o que consta nos Processos nºs.  
23057.001221.2020-05, 23057.001222.2020-41, de 14 de fevereiro de 2020,
D E L I B E R A:
I – APROV AR, na forma dos anexos, para serem submetidos ao Conselho Superior, o Proje to Pedagógico e o Projeto de
Autorização de Funcionamento do Curso Superior de Tecnologia em Sistemas para Internet, na modalidade a distancia, no Campus
Natal-Zona Leste deste Instituto Federal.' metadata={'source': 'd

In [3]:
print (f"Quantidade de chunks: {len(texts)} ")

Quantidade de chunks: 623 


In [4]:
#!pip install faiss-cpu
#pip install faiss-gpu

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.from_documents(texts,  OllamaEmbeddings(model="mxbai-embed-large"))

/var/folders/fh/g5fgxqq513n014czxdxy7sfc0000gn/T/ipykernel_84338/2162803906.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = FAISS.from_documents(texts,  OllamaEmbeddings(model="mxbai-embed-large"))


In [6]:
query = "Qual o nome do curso?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Curso Superior de Tecnologia em Sistemas para Internet, na modalidade a distância 
                                                                              IFRN, 2022  
 
91 
 
3. TAYLOR, A. G. SQL para leigos. 1. Ed. Rio de Janeiro: Starlin, 2016. 
 
Bibliografia Complementar 
1. DATE, C.J. Introdução a Sistemas de Banco de Dados. 1.Ed. Rio de Janeiro: Campus, 2004. 
2. HEUSER, CARLOS A. Projeto de Banco de Dados. 6.Ed. Porto Alegre: Bookman, 2009. 
3. DAMAS, L. SQL ʹ  Structure Query Language. 6. ed. Rio de Janeiro: LTC, 2007. 
4. FORTA, B. SQL in 10 Minutes. 4 Ed. Indianapolis: Sams, 2012. 
5. MYSQL. MySQL 8.0 Reference Manual. Disponível em: https://dev.mysql.com/doc/refman/8.0/en/. Acesso 
em 25 de junho de 2019. 
 
Software(s) de Apoio: 
● DBDesigner, brModelo; 
● Serviço MySQL, MySQL Workbench; 
● Serviço PostgreSQL, pgAdmin; 
● Serviço SQL Server, Microsoft SQL Server Express Edition. 
● Plataforma Moodle


In [7]:
# Perform similarity search to get the top 5 most relevant chunks (by similarity)
docs = db.similarity_search(query, k=5)

# Print the content of each of the top 5 chunks
for i, doc in enumerate(docs):
    print(f"Chunk {i + 1}: {doc.page_content}")

Chunk 1: Curso Superior de Tecnologia em Sistemas para Internet, na modalidade a distância 
                                                                              IFRN, 2022  
 
91 
 
3. TAYLOR, A. G. SQL para leigos. 1. Ed. Rio de Janeiro: Starlin, 2016. 
 
Bibliografia Complementar 
1. DATE, C.J. Introdução a Sistemas de Banco de Dados. 1.Ed. Rio de Janeiro: Campus, 2004. 
2. HEUSER, CARLOS A. Projeto de Banco de Dados. 6.Ed. Porto Alegre: Bookman, 2009. 
3. DAMAS, L. SQL ʹ  Structure Query Language. 6. ed. Rio de Janeiro: LTC, 2007. 
4. FORTA, B. SQL in 10 Minutes. 4 Ed. Indianapolis: Sams, 2012. 
5. MYSQL. MySQL 8.0 Reference Manual. Disponível em: https://dev.mysql.com/doc/refman/8.0/en/. Acesso 
em 25 de junho de 2019. 
 
Software(s) de Apoio: 
● DBDesigner, brModelo; 
● Serviço MySQL, MySQL Workbench; 
● Serviço PostgreSQL, pgAdmin; 
● Serviço SQL Server, Microsoft SQL Server Express Edition. 
● Plataforma Moodle
Chunk 2: Curso Superior de Tecnologia em Sistemas para Inte

# Explorando retriever

In [8]:
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA

# Create the LLM model and retriever
model = OllamaLLM(model="llama3.2:latest")

# recuperando o retriever com 5 documentos
retriever = db.as_retriever(search_kwargs={"k": 5})
#retriever = db.as_retriever()

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever, chain_type="stuff")

In [9]:
# Example usage of the RetrievalQA chain
response = qa_chain.invoke(query)
print("QA Response:", response)

QA Response: {'query': 'Qual o nome do curso?', 'result': 'O nome do curso é Superior de Tecnologia em Sistemas para Internet.'}
